In [0]:
!pip install transformers tqdm boto3 requests regex -q

     |████████████████████████████████| 573kB 6.3MB/s 
     |████████████████████████████████| 1.0MB 40.3MB/s 
     |████████████████████████████████| 3.7MB 36.2MB/s 
     |████████████████████████████████| 890kB 30.4MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My\ Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [0]:
import pandas as pd

df_test = pd.read_excel("stock_data/stockdata_withdo_testROI.xlsx")


In [0]:
import pandas as pd
from pandas import DataFrame
l1 = []
l2 = []
l3 = []
l4 = []
l5 = []
l6 = []
l7 = []

for i, row in df_test.iterrows():
    content = row['content'][:58]
    l1.extend([content])
    l2.extend([row['labeling']])
    l3.extend([row['start_date']])
    l4.extend([str(row['start_price'])])
    l5.extend([row['last_date']])
    l6.extend([str(row['last_price'])])
    l7.extend([row['code']])
    
df_test = DataFrame({'content': l1, 'labeling': l2, 'start_date':l3, 'start_price':l4, 'last_date':l5, 'last_price':l6, 'code':l7})

In [0]:
# 去除不必要欄位

df_test = df_test.loc[:, ['content', 'labeling', 'start_date', 'start_price', 'last_date', 'last_price', 'code']]
df_test.columns = ['content', 'labeling', 'start_date', 'start_price', 'last_date', 'last_price', 'code']
df_test.to_csv("test.tsv", sep="\t", index=False)


In [0]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""

from torch.utils.data import Dataset
from transformers import BertTokenizer
import torch

PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    
class StockDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test", "dev"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test":
            content, label = self.df.iloc[idx, :2].values
            label_id = label
            label_tensor = torch.tensor(label_id)
        else:
            content, label = self.df.iloc[idx, :2].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = label
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(content)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
                
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a , 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len

In [0]:
model = torch.load('Bert_model/Bert50_50_50.pkl')

In [0]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式

GPU 跑會有 cuda runtime error 的問題，所以先使用 CPU 跑不會有狀況!

"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()

                tp += (labels * pred).sum().to(torch.float32)
                tn += ((1 - labels) * (1 - pred)).sum().to(torch.float32)
                fp += ((1 - labels) * pred).sum().to(torch.float32)
                fn += (labels * (1 - pred)).sum().to(torch.float32)
                                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total

        #epsilon = 1e-7
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        fscore = (2 * precision * recall) / (precision + recall)
        return predictions, acc, precision, recall, fscore    
    return predictions
  

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
# _, acc, precision, recall, fscore = get_predictions(model, trainloader, compute_acc=True)
# print("classification acc:", acc)

device: cuda:0


In [0]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 訓練集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

In [0]:
testset = StockDataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=32, 
                        collate_fn=create_mini_batch)

predictions = get_predictions(model, testloader)

df = pd.DataFrame({"pre_labeling": predictions.tolist()})

df_pred = pd.concat([testset.df.loc[:, ["code"]],
                     testset.df.loc[:, ["start_date"]], 
                     testset.df.loc[:, ["start_price"]],
                     testset.df.loc[:, ["last_date"]],
                     testset.df.loc[:, ["last_price"]], 
                     df.loc[:, 'pre_labeling']], axis=1)
# df_pred.to_excel('stock_data/BERT_prediction.xlsx', index=False)

In [0]:
# dfs = pd.read_excel('stock_data/BERT_prediction.xlsx')

dfs = df_pred

# 將code跟日期做排序
dfs.sort_values(['code', 'start_date'],inplace=True)

# 將 0 換成 -1 --> 好計算
dfs['pre_labeling'].replace(0, -1, inplace=True)

In [0]:
# 將 相同股票公司 與 發布新聞 做正負向相加
calculated = dfs.groupby(['code', 'start_date', 'start_price', 'last_date', 'last_price'], as_index=False)['pre_labeling'].sum()

# 去除掉預測分數為0的值，因為不能確定是會上漲還是下跌
calculated = calculated[~(calculated == 0).any(axis=1)]

# 預測數值 > 0 的為正向； < 0 的則為負向
calculated['pre_labeling'].replace([1,2,3,4,5,6,7,8,9,10], 1, inplace=True)
calculated['pre_labeling'].replace([-1,-2,-3,-4,-5,-6,-7,-8,-9,-10], 0, inplace=True)
calculated

,code,start_date,start_price,last_date,last_price,pre_labeling
0,1101,2019/06/03,43.40,2019/06/04,43.45,1
3,1101,2019/06/10,43.85,2019/06/11,43.75,1
4,1101,2019/06/11,44.00,2019/06/12,43.95,1
5,1101,2019/06/12,43.75,2019/06/13,43.95,1
6,1101,2019/06/13,43.85,2019/06/14,43.90,1
...,...,...,...,...,...,...
526,9910,2019/06/20,249.00,2019/06/21,252.50,1
527,9910,2019/06/21,245.00,2019/07/31,208.00,1
528,9910,2019/06/24,245.00,2019/06/25,243.00,0
529,9910,2019/06/26,243.50,2019/06/27,240.50,1


In [0]:
# 將每支股票的最後一筆資料做標記
dfs = calculated.groupby('code').tail(1)
dfs_index = dfs.index.to_list()

df['last_data'] = 0

for i, rows in calculated.iterrows():
    if i in set(dfs_index):
        last_data = int(1)
    else:
        last_data = int(0)

    calculated.loc[i, 'last_data'] = last_data

calculated

,code,start_date,start_price,last_date,last_price,pre_labeling,last_data
0,1101,2019/06/03,43.40,2019/06/04,43.45,1,0.0
3,1101,2019/06/10,43.85,2019/06/11,43.75,1,0.0
4,1101,2019/06/11,44.00,2019/06/12,43.95,1,0.0
5,1101,2019/06/12,43.75,2019/06/13,43.95,1,0.0
6,1101,2019/06/13,43.85,2019/06/14,43.90,1,0.0
...,...,...,...,...,...,...,...
526,9910,2019/06/20,249.00,2019/06/21,252.50,1,0.0
527,9910,2019/06/21,245.00,2019/07/31,208.00,1,0.0
528,9910,2019/06/24,245.00,2019/06/25,243.00,0,0.0
529,9910,2019/06/26,243.50,2019/06/27,240.50,1,0.0


In [0]:
cat_dict = {
    '2330':'0.4016',
    '2317':'0.0521',
    '2454':'0.0317',
    '2412':'0.0242',
    '1301':'0.0235',
    '3008':'0.0229',
    '2891':'0.0209',
    '1303':'0.0199',
    '1216':'0.0192',
    '2882':'0.0184',
    '2886':'0.0184',
    '2881':'0.0175',
    '2884':'0.0169',
    '2308':'0.0167',
    '1326':'0.0143',
    '2002':'0.0143',
    '3711':'0.0129',
    '2892':'0.0121',
    '1101':'0.0118',
    # '202005TX':'0.0117',
    '2885':'0.011',
    '5880':'0.0108',
    '3045':'0.01',
    '2382':'0.0099',
    '2303':'0.0098',
    '2880':'0.0098',
    '2912':'0.0096',
    '2207':'0.0095',
    '2474':'0.0088',
    '2357':'0.0082',
    '5876':'0.0081',
    '2327':'0.0078',
    '4938':'0.0075',
    '2801':'0.0072',
    '2887':'0.0072',
    '2890':'0.0071',
    '5871':'0.0071',
    '2883':'0.0069',
    '6505':'0.0066',
    '4904':'0.0062',
    '1102':'0.0058',
    '1402':'0.0058',
    '2395':'0.0058',
    '2888':'0.0057',
    '2301':'0.0054',
    '9904':'0.0045',
    '2633':'0.0039',
    '2105':'0.0036',
    '2823':'0.0034',
    '9910':'0.0031',
    '2408':'0.0029',
}

In [0]:
# earned = 0
# hold = 0
# budget = 1000000

# for i, row in calculated.iterrows():
#     if hold == 0:
#         if row['pre_labeling'] == 1:
#             hold_price = round(row['start_price'], 2)
#             hold = 1
            
#     elif hold == 1:
#         if row['pre_labeling'] == 0:
#             earned += round(row['last_price'] - hold_price, 2)
#             hold = 0
#             hold_price = 0

#     if row['last_data'] == 1 and hold == 1:
#         stock_df = pd.read_excel(('stock_data/stockprice/{}.xlsx').format(row['code']))
#         price = stock_df[stock_df['日期'] == '2019/06/28']['開盤'].values[0]
#         earned += round(price - hold_price, 2)
#         hold = 0
#         hold_price = 0

In [0]:
earned = 0
hold = 0
budget = 1000000
percent = 0

for i, row in calculated.iterrows():
    code = row['code']
    if hold == 0:
        if row['pre_labeling'] == 1:
            percent = [ float(cat) for cat_id, cat in cat_dict.items() if int(code) == int(cat_id) ].pop()
            hold_price = round(row['start_price'] * percent, 4)
            hold = 1
            
    elif hold == 1:
        if row['pre_labeling'] == 0:
            percent = [ float(cat) for cat_id, cat in cat_dict.items() if int(code) == int(cat_id) ].pop()
            earned += round((row['last_price'] * percent) - hold_price, 4)
            hold = 0
            hold_price = 0

    if row['last_data'] == 1 and hold == 1:
        stock_df = pd.read_excel(('stock_data/stockprice/{}.xlsx').format(row['code']))
        price = stock_df[stock_df['日期'] == '2019/06/28']['開盤'].values[0]
        percent = [ float(cat) for cat_id, cat in cat_dict.items() if int(code) == int(cat_id) ].pop()
        earned += round((price * percent) - hold_price, 4)
        hold = 0
        hold_price = 0

earned

7.7382